In [ ]:
!pip install -q transformers
!pip install evaluate

import huggingface_hub
from transformers import AutoModelWithLMHead, AutoTokenizer
import torch
from transformers import TrainingArguments, Trainer

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
tokenizer.padding_side = "left"
model = AutoModelWithLMHead.from_pretrained("microsoft/DialoGPT-medium") # 모델 불러오기

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:2020: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
#huggingface에서 model및 data 불러오기
from sentence_transformers import SentenceTransformer  # 문장을 벡터로 변환하는 모델
from transformers import AutoTokenizer  # 토큰화를 위한 도구
from datasets import load_dataset
import huggingface_hub
# 필요한 라이브러리 호출
import torch
import json
import pandas as pd
import re

# 데이터 선언 및 확인
# load로 다운받는 방법 / wget 다운
#https://huggingface.co/datasets/{데이터셋 경로}/resolve/main/{파일 경로}

#test dataset
!wget https://huggingface.co/datasets/amaydle/npc-dialogue/resolve/main/data/test-00000-of-00001-0408c6b1dfcc3c77.parquet -O test.parquet
#train dataset
!wget https://huggingface.co/datasets/amaydle/npc-dialogue/resolve/main/data/train-00000-of-00001-4eeea4877d4ce970.parquet -O train.parquet

# wget으로 다운로드 받은 parquet 파일 읽기
test_df = pd.read_parquet("test.parquet")
train_df = pd.read_parquet("train.parquet")

with open("dialogpt_test_prompt.jsonl", "w", encoding="utf-8") as f_out:
    # 4) 행(row) 단위 반복
    for _, row in test_df.iterrows():
        # 4-1) DialoGPT 프롬프트 구성 — train 때 썼던 포맷 그대로
        prompt = (
            f"Name: {row['Name']}\n"
            f"Biography: {row['Biography']}\n"
            f"Question: {row['Query']}\n"
            f"Answer:"
        )
        # 4-2) NPC 실제 답변
        completion = f"{row['Response']}"

        # 4-3) JSON 객체 만들기
        json_obj = {"prompt": prompt, "completion": completion}

        # 4-4) 한 줄(JSONL)로 기록
        #      ensure_ascii=False → 한글 그대로 저장
        f_out.write(json.dumps(json_obj, ensure_ascii=False) + "\n")

print("✅ test split → JSONL 저장 완료: dialogpt_test_prompt.jsonl")

#DialoGPT tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
print(tokenizer)
# 전처리 함수 정의
def clean_text(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# formatted_texts & targets 생성
inputs = []
targets = []

for idx, row in train_df.iterrows():
    # 각 필드 전처리
    name = clean_text(str(row['Name']))
    bio = clean_text(str(row['Biography']))
    query = clean_text(str(row['Query']))
    response = clean_text(str(row['Response']))

    # input prompt 구성
    input_text = (
        f"Name: {name}\n"
        f"Biography: {bio}\n"
        f"Question: {query}\n"
        f"Answer:"
    )

    # 리스트에 추가
    inputs.append(input_text)
    targets.append(response)

 #Tokenization & Encoding (DialoGPT 학습용)
# - labels도 input_ids 기반으로 생성
tokenizer.pad_token = tokenizer.eos_token

encodings = tokenizer(
    inputs,
    text_target=targets,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# 최종 데이터셋 준비 (tensor 형)

input_ids = encodings['input_ids']
attention_mask = encodings['attention_mask']
labels = encodings['labels']
print(labels)
print("\n[input_ids shape]:", input_ids.shape)
print("[attention_mask shape]:", attention_mask.shape)
print("[labels shape]:", labels.shape)

#  DataLoader로 연결 가능 (예시)

from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(input_ids, attention_mask, labels)
loader = DataLoader(dataset, batch_size=8, shuffle=True)
print(loader)
print("\n[DataLoader 준비 완료]")

--2025-07-02 08:09:41--  https://huggingface.co/datasets/amaydle/npc-dialogue/resolve/main/data/test-00000-of-00001-0408c6b1dfcc3c77.parquet
Resolving huggingface.co (huggingface.co)... 18.172.134.24, 18.172.134.88, 18.172.134.4, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/bb/e5/bbe5bffdb17c314c4f037cb50945b54c27a7885a7e5150cd833328f4b500a267/d375063db7691154e53f6a30048c77c74debaaf8aed92c7cbe16d46ff58ea2e6?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27test-00000-of-00001-0408c6b1dfcc3c77.parquet%3B+filename%3D%22test-00000-of-00001-0408c6b1dfcc3c77.parquet%22%3B&Expires=1751447382&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MTQ0NzM4Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9iYi9lNS9iYmU1YmZmZGIxN2MzMTRjNGYwMzdjYjUwOTQ1YjU0YzI3YTc4ODVhN2U1MTUwY2Q4MzMzMjhmNGI1MDBhMjY3L2QzNzUwNjNkYjc2OT

In [ ]:
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import torch
from transformers import DataCollatorForLanguageModeling

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # BLEU 계산
    bleu = evaluate.load("bleu")
    decoded_preds = [pred.strip().split() for pred in decoded_preds]
    decoded_labels = [[label.strip().split()] for label in decoded_labels]

    bleu_score = bleu.compute(predictions=decoded_preds, references=decoded_labels)

    return {"eval_bleu": bleu_score["bleu"]}


data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# 데이터 셋을 HuggingFace Dataset 으로 변환
dataset_dict = Dataset.from_dict({
    "input_ids": input_ids,
    "attention_mask": attention_mask,
    "labels": labels
})
tokenizer.pad_token = tokenizer.eos_token
train_data,val_data = dataset_dict.train_test_split(test_size=0.2).values() # 전체 데이터 셋에서 전체의 20%를 Validation set으로 설정
# .train_test_split()은 내부적으로 random.shuffle()을 하여 랜덤하게 분활
# .values()는 결과가 딕셔너리 형태이므로 이를 튜플처럼 꺼내오는 것

training_args = TrainingArguments(
    output_dir="./dialogpt-custom_s2",
    num_train_epochs=10,
    per_device_train_batch_size=2,
    logging_steps=50,
    save_steps=500,
    save_total_limit=3,
    fp16=True,
    eval_strategy="steps",         # 매 epoch마다 validation 평가
    save_strategy="steps",               # 매 epoch마다 저장
    load_best_model_at_end=True,         # 가장 성능 좋은 모델 자동 로드
    metric_for_best_model="loss",        # 평가 기준: loss
    greater_is_better=False ,# 낮을수록 좋은 경우 (loss)
    learning_rate=2e-5,
    weight_decay=0.01
)


# 그대로 Trainer에 넣기
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(
        early_stopping_patience=10)           # ★ 추가
    ]
)

trainer.train()  # 모델 훈련
trainer.save_model("./my_saved_model333")  # 훈련 완료 후 모델 저장
tokenizer.save_pretrained("./my_saved_model333")  # 토크나이저도 함께 저장



/tmp/ipython-input-59-2990912935.py:55: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
50,0.005700,0.397943
100,0.030800,0.387336
150,0.028900,0.391151
200,0.032500,0.370995
250,0.020800,0.381414
300,0.019400,0.378261
350,0.018000,0.357028
400,0.021900,0.380796
450,0.016700,0.379831
500,0.065600,0.348908


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


('./my_saved_model333/tokenizer_config.json',
 './my_saved_model333/special_tokens_map.json',
 './my_saved_model333/chat_template.jinja',
 './my_saved_model333/vocab.json',
 './my_saved_model333/merges.txt',
 './my_saved_model333/added_tokens.json',
 './my_saved_model333/tokenizer.json')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# 1. 모델 경로 설정
model_path = "/content/my_saved_model333"  # 학습 시 output_dir로 지정했던 경로

# 2. 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# pad_token 설정 (GPT 계열 모델은 pad_token이 기본 정의되어 있지 않음)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# 3. 디바이스 설정 및 모델 준비
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# 4. 대화 이력 관리 파라미터
MAX_HISTORY_TOKENS = 512    # 유지할 최대 컨텍스트 토큰 길이
MAX_NEW_TOKENS = 100        # 한 번에 생성할 최대 토큰 수

print("🤖 DialoGPT 챗봇과 대화를 시작하세요! ('exit' 입력 시 종료)\n")

chat_history_ids = None

while True:
    user_input = input(">> User: ")
    if user_input.strip().lower() in ["exit", "quit"]:
        print("👋 대화를 종료합니다.")
        break

    # 사용자 입력 토크나이즈
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors="pt").to(device)

    # 이전 대화 이력과 연결 및 슬라이딩 윈도우 적용
    if chat_history_ids is None:
        bot_input_ids = new_input_ids
    else:
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1)
        # 최대 히스토리 토큰 수 초과 시 뒤쪽만 유지
        if bot_input_ids.shape[-1] > MAX_HISTORY_TOKENS:
            bot_input_ids = bot_input_ids[:, -MAX_HISTORY_TOKENS:]

    # 응답 생성 (그래디언트 비활성화)
    with torch.no_grad():
        chat_history_ids = model.generate(
            bot_input_ids,
            max_new_tokens=MAX_NEW_TOKENS,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7,
        )

    # 생성된 응답 디코딩 (CPU로 이동 후 디코딩)
    response_ids = chat_history_ids[:, bot_input_ids.shape[-1]:][0].cpu()
    reply = tokenizer.decode(response_ids, skip_special_tokens=True)
    print("🤖 DialoGPT:", reply)


🤖 DialoGPT 챗봇과 대화를 시작하세요! ('exit' 입력 시 종료)

>> User: hello
🤖 DialoGPT: Hello! How are you today?
>> User: what is your favorite color?
🤖 DialoGPT: I'm a shade of brown and I love to dance and sing. How about you?
>> User: i like brown too!!
🤖 DialoGPT: Have you ever been to a different galaxy than you are now?
>> User: no... 
🤖 DialoGPT: Okay then. Have you ever been on a spaceship other than Earth?
>> User: no........
🤖 DialoGPT: Well I guess I should have known better than to trust a person who has never been on a spaceship. Have you ever been on a different planet than you are now?


KeyboardInterrupt: Interrupted by user

In [ ]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import evaluate
from tqdm import tqdm
model_path = "./my_saved_model333"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to("cuda")
model.eval() # 모델 추론 상태로 전환

bleu = evaluate.load("bleu")

test_data=[]
with open("/content/dialogpt_test_prompt.jsonl","r",encoding="utf-8") as f:
  for line in f:
      test_data.append(json.loads(line))

batch_size=16
predictions,references = [],[]

for i in tqdm(range(0,len(test_data),batch_size)):
  batch = test_data[i:i+batch_size]
  prompts = [item["prompt"] + tokenizer.eos_token for item in batch]
  target_refs = [[item["completion"]] for item in batch]

  encoded = tokenizer(
      prompts,
      return_tensors="pt", # 모델에 바로 넣을 수 있는 입력을 만들기 위해 텐서 형태로 변환
      truncation=True, # 입력 테스트가 너무 늘 경우 자름
      max_length=128,
      padding=True
  ).to(model.device)

  # 응답 생성
  with torch.no_grad():
    output_ids = model.generate(
        input_ids=encoded["input_ids"],
        attention_mask=encoded["attention_mask"], # 어떤 부분 주의 계산에 포함 시킬지
        max_length=300,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True, # 텍스트 생성 시 확률 기반 샘플링 활성화
        # 다음 토큰을 생성할 때 가장 확률이 높은 하나만 고르지 않고 확률 분포에 따라 무작위 샘플
        top_k=50,
        top_p=0.95
    )
  generated = tokenizer.batch_decode(output_ids, skip_special_tokens=True)

  for pred, ref in zip(generated, target_refs):
      if pred and ref and ref[0].strip():
          predictions.append(pred.strip())
          references.append([ref[0].strip()])

score = bleu.compute(predictions=predictions, references=references)
print("\n BLEU score:", score["bleu"])
